In [2]:
import os
import pymongo
import pandas as pd
import re
import pickle
import numpy as np
import pandas as pd
import html2text as h2t
import requests
import base64
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tag import pos_tag
from nltk import PorterStemmer
import string
import PyPDF2
import io
path = os.path.abspath(os.getcwd())

#US Govt
#state_gov_nm = 'US'
#state_gov_id = 52

#NH
state_gov_nm = 'NH'
state_gov_id = 29

mongo_username= 'CEDC_Quigley'
mongo_password= 'lpP9pCcDxQ2Y0tv8'
mongo_server = 'mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
mongo_authSource = 'admin'
mongo_authMechanism = 'SCRAM-SHA-1'
mongo_database = 'legislative_documents'
mongo_people_collection = 'legiscan_people_data'
mongo_billmeta_collection = 'legiscan_bill_metadata'
mongo_billtext_collection = 'legislative_texts'

client = pymongo.MongoClient(mongo_server
                        ,username=mongo_username
                        ,password=mongo_password
                        ,authSource=mongo_authSource
                        ,authMechanism=mongo_authMechanism)

_legtext09_20 = '{}/{}/legtext09_20.corpus'.format(path,state_gov_nm)

_raw_bill_metadata09_20 = '{}/{}/raw_bill_metadata09_20.list'.format(path,state_gov_nm)



#This is used to determine the line after which the text begins
#textLinetoStartAfter'SESSION'
textLinetoStartAfter = 'In the Year of Our Lord'
usetextLinetoStartAfter = False

In [3]:
database = client[mongo_database]
collection = database[mongo_billmeta_collection]

bill_data = []

for bl in collection.find({'state':state_gov_nm}).batch_size(30):
    bill_data.append(bl)


In [4]:
bill_ids = []
for bill in bill_data:
    bill_ids.append(bill['bill_id'])

In [5]:
with open(_raw_bill_metadata09_20, 'wb') as f:
    pickle.dump(bill_data, f)

In [6]:
database = client[mongo_database]
collection = database[mongo_billtext_collection]

legtext_corpus = [[0,0,['dummy','entry']]]

count = 0

legtext_corpus = []
for bl in collection.find({'bill_id': {'$in': bill_ids}}).batch_size(30):
    #try:
    if count%1000 == 0:
        print(count)
    count+=1
        #check the mime of the bill text and extract as necessary
    if bl['mime'] == 'text/html':
        #base64_bytes = bl['doc'].encode('unicode_escape')
        message_bytes = base64.b64decode(bl['doc'])#base64_bytes)
        message = message_bytes.decode('unicode_escape')
    elif bl['mime'] == 'application/pdf':
        message = base64.b64decode(bl['doc'])

    #base64_bytes = bl['doc'].encode('unicode_escape')
    #message_bytes = base64.b64decode(bl['doc'])#base64_bytes)
    #message = message_bytes.decode('unicode_escape')
    legtext_corpus.append([bl['bill_id'], bl['doc_id'], message, bl['mime']])
    #except:
    #    pass


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [7]:
#process the downloaded data
print(len(legtext_corpus))
for ind, document in enumerate(legtext_corpus):
    if ind%1000 == 0:
        print(ind)
    
    text = document[2]
    
    rawbillText = ''

    #check the mime of the bill text and extract as necessary
    if document[3] == 'text/html':
        rawbillText = h2t.html2text(text)
    
    elif document[3] == 'application/pdf':
        
        pdftext = io.BytesIO(bytes(text))

        pdfReader = PyPDF2.PdfFileReader(pdftext)
        for pdfPage in pdfReader.pages:

            rawbillText += pdfPage.extractText()
    
    billText=''
    
    if usetextLinetoStartAfter:
        i=False
    else:
        i=True
    
    for l in rawbillText.split('\n'):
        #look through every line, looking for textLinetoStartAfter
        if i:
            billText=billText+str(l)+' '
        if textLinetoStartAfter in l:
            i=True
        
            
    #tokenize the text
    tokens = word_tokenize(billText)
    #create a list of parts of speech tags for the tokens
    tags = pos_tag(tokens)
    # convert to lower case
    tokens = [w.lower() for w in tokens]

    # remove punctuation
    table = str.maketrans('', '', string.punctuation)
    strippedBillText = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    wordsInBill = [word for word in strippedBillText if word.isalpha()]

    #define the single character 'words' that need to be removed
    singleCharacters = ['a','b','c','d','e','f','g','h','i','j',\
                        'k','l','m','n','o','p','q','r','s','t',\
                        'u','v','w','x','y','z']

    #grab the list of words that were determined to be proper nouns
    #ProperNouns = [word.lower() for word, tag in tags if tag == 'NNP' or tag == 'NNPS' or tag == 'IN']

    #define stop words
    stop_words = set(stopwords.words('english'))
    # filter out stop words, single letter words, and proper nouns
    wordsInBill = [w for w in wordsInBill if not w in stop_words]
    wordsInBill = [w for w in wordsInBill if not w in singleCharacters]
    #wordsInBill = [w for w in wordsInBill if not w in ProperNouns]

    #roll words up to simpler, more categorical forms to make comparison easier
    # stemming of words
    #porter = PorterStemmer()
    #wordsInBill = [porter.stem(word) for word in wordsInBill]

    #create the corpus object
    
    legtext_corpus[ind][2] =  wordsInBill
    #legdoc = [bill_id,doc_id,wordsInBill]
    #legtext_corpus.append(legdoc)

17802
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [8]:
with open(_legtext09_20,'wb') as f:
    pickle.dump(legtext_corpus, f)